# Outline

  conda create -n ancient_dna_env python=3.8 biopython pandas matplotlib numpy jupyter ipython scipy seaborn -y


1. Creating a Directory: Checks if an 'output' directory exists; if not, it creates one.

2. Loading Data: Loads AmtDB metadata and sequence IDs from a specified FASTA file.

3. Finding Missing Sequences: Identifies sequences present in AmtDB metadata but missing from the FASTA file.

4. Loading a Mitochondrial Dataset: Loads a mitochondrial (mt) dataset's sequence IDs and metadata from specified files.

5. Extracting and Saving Sequences: Extracts sequences that match specified IDs from a FASTA file and saves them to a new file.

6. Matching and Saving Metadata: Matches metadata for specified IDs and saves it to a new CSV file.

7. Main Workflow:
    * Initializes the process by creating directories and loading initial datasets.
    * Identifies sequences missing in the AmtDB dataset but available in the mitochondrial dataset.
    * For sequences found, extracts these sequences and their metadata, saving them to the 'output' directory.
    * Additionally, identifies sequences available in the mitochondrial dataset but not in the AmtDB, extracting and saving these as well.
    * Completes the execution by indicating the results are in the 'output' directory.

In [5]:
from Bio import SeqIO
import pandas as pd
import os

def create_directories():
    """
    Creates 'output' directory if it does not exist.
    """
    if not os.path.exists('output'):
        print("Creating 'output' directory...\n")
        os.makedirs('output')
    else:
        print("'output' directory already exists.\n")

def load_data(metadata_file, fasta_file):
    """
    Load AmtDB metadata and sequence IDs from the FASTA file.

    Args:
        metadata_file (str): Path to the AmtDB metadata file (e.g. 'amtdb_metadata.csv')
        fasta_file (str): Path to the AmtDB FASTA file (e.g. 'amtdb_1621-samples_7f_a0pkh.fasta')

    Returns:
        meta_amtDB (DataFrame): AmtDB metadata
        ids_seq_fasta (list): List of sequence IDs from the FASTA file (e.g. ['seq1', 'seq2', ...]
    """
    print(f"Loading AmtDB metadata from '{metadata_file}' and sequence IDs from '{fasta_file}'...")
    meta_amtDB = pd.read_csv(metadata_file, sep=',', header=0)
    ids_seq_fasta = [seq_record.id for seq_record in SeqIO.parse(fasta_file, "fasta")]
    print(f"Loaded AmtDB metadata with {len(meta_amtDB)} records and {len(ids_seq_fasta)} sequences.\n")
    return meta_amtDB, ids_seq_fasta

def find_missing_sequences(meta_amtDB, ids_seq_fasta):
    """
    Identifies sequences present in metadata but missing from the FASTA file.

    Args:
        meta_amtDB (DataFrame): AmtDB metadata
        ids_seq_fasta (list): List of sequence IDs from the FASTA file (e.g. ['seq1', 'seq2', ...]

    Returns:
        missing_ids (list): List of sequence IDs present in metadata but missing from the FASTA file
    """
    print("Identifying sequences present in metadata but missing from the FASTA file...")
    amtDB_ids = set(meta_amtDB['identifier'])
    fasta_ids = set(ids_seq_fasta)
    missing_ids = list(amtDB_ids.difference(fasta_ids))
    print(f"Found {len(missing_ids)} missing sequences.\n")
    return missing_ids

def load_mt_dataset(fasta_file, anno_file):
    """
    Load the Reich mt dataset and metadata.

    Args:
        fasta_file (str): Path to the Reich mt dataset FASTA file (e.g. 'mtdna_reich.fasta')
        anno_file (str): Path to the Reich mt dataset metadata file (e.g. 'v54.1.p1_1240K_public.anno')

    Returns:
        ids_mt_dataset (list): List of sequence IDs from the FASTA file (e.g. ['seq1', 'seq2', ...]
        meta_mt_dataset (DataFrame): Reich mt dataset metadata
    """
    print(f"Loading 'Reich mt dataset' from '{fasta_file}' and metadata from '{anno_file}'...")
    ids_mt_dataset = [seq_record.id for seq_record in SeqIO.parse(fasta_file, "fasta")]
    meta_mt_dataset = pd.read_csv(anno_file, sep='\t', header=0, low_memory=False)
    print(f"Loaded 'Reich mt dataset' with {len(ids_mt_dataset)} sequences and metadata with {len(meta_mt_dataset)} records.\n")
    return ids_mt_dataset, meta_mt_dataset

def extract_and_save_sequences(fasta_file, ids, output_file):
    """
    Extracts sequences matching the specified IDs and saves them to a new FASTA file.

    Args:
        fasta_file (str): Path to the input FASTA file
        ids (list): List of sequence IDs to extract
        output_file (str): Path to the output FASTA file
    """
    print(f"Extracting sequences matching the specified IDs from '{fasta_file}'...")
    sequences = [seq_record for seq_record in SeqIO.parse(fasta_file, "fasta") if seq_record.id in ids]
    SeqIO.write(sequences, output_file, "fasta")
    print(f"Saved {len(sequences)} sequences to '{output_file}'.\n")

def match_and_save_metadata(meta_mt_dataset, ids, output_file, id_column_name):
    """
    Matches metadata for the specified IDs and saves it to a new CSV file.

    Args:
        meta_mt_dataset (DataFrame): Reich mt dataset metadata
        ids (list): List of sequence IDs to match
        output_file (str): Path to the output CSV file
        id_column_name (str): Name of the column containing sequence IDs in the metadata
    """
    print(f"Matching metadata for the specified IDs and saving to '{output_file}'...")
    matched_metadata = meta_mt_dataset[meta_mt_dataset[id_column_name].isin(ids)]
    matched_metadata.to_csv(output_file, sep=',', index=False)
    print(f"Saved metadata for {len(matched_metadata)} sequences to '{output_file}'.\n")

def main():
    """
    Main pipeline function.
    """
    create_directories()
    meta_amtDB, ids_seq_fasta_amtDB = load_data('data/amtDB/amtdb_metadata.csv', 'data/amtDB/amtdb_1621-samples_7f_a0pkh.fasta')
    ids_mt_dataset, meta_mt_dataset = load_mt_dataset('data/mitogenomes_reich/mtdna_reich.fasta', 'data/mitogenomes_reich/v54.1.p1_1240K_public/v54.1.p1_1240K_public.anno')

    missing_ids = find_missing_sequences(meta_amtDB, ids_seq_fasta_amtDB)
    available_in_mt_dataset = set(ids_mt_dataset).intersection(set(missing_ids))

    id_column_name = 'Master ID'
    if available_in_mt_dataset:
        extract_and_save_sequences('data/mitogenomes_reich/mtdna_reich.fasta', available_in_mt_dataset, f'output/sequences_missing_in_AmtDB_{len(available_in_mt_dataset)}.fasta')
        match_and_save_metadata(meta_mt_dataset, available_in_mt_dataset, f'output/metadata_for_sequences_missing_in_AmtDB.csv', id_column_name)

    not_in_amtDB = set(ids_mt_dataset) - set(meta_amtDB['identifier'])
    if not_in_amtDB:
        extract_and_save_sequences('data/mitogenomes_reich/mtdna_reich.fasta', not_in_amtDB, f'output/sequences_not_present_in_AmtDB_{len(not_in_amtDB)}.fasta')
        match_and_save_metadata(meta_mt_dataset, not_in_amtDB, f'output/metadata_for_sequences_not_present_in_AmtDB.csv', id_column_name)
        
    print("Pipeline execution complete. Check the 'output' directory for results.\n")

    
    
if __name__ == "__main__":
    main()


'output' directory already exists.

Loading AmtDB metadata from 'data/amtDB/amtdb_metadata.csv' and sequence IDs from 'data/amtDB/amtdb_1621-samples_7f_a0pkh.fasta'...
Loaded AmtDB metadata with 2541 records and 1621 sequences.

Loading 'Reich mt dataset' from 'data/mitogenomes_reich/mtdna_reich.fasta' and metadata from 'data/mitogenomes_reich/v54.1.p1_1240K_public/v54.1.p1_1240K_public.anno'...
Loaded 'Reich mt dataset' with 4122 sequences and metadata with 16388 records.

Identifying sequences present in metadata but missing from the FASTA file...
Found 920 missing sequences.

Extracting sequences matching the specified IDs from 'data/mitogenomes_reich/mtdna_reich.fasta'...
Saved 404 sequences to 'output/sequences_missing_in_AmtDB_404.fasta'.

Matching metadata for the specified IDs and saving to 'output/metadata_for_sequences_missing_in_AmtDB.csv'...
Saved metadata for 431 sequences to 'output/metadata_for_sequences_missing_in_AmtDB.csv'.

Extracting sequences matching the specified

In [4]:
# load the output fasta file and check the number of sequences
# load the output metadata file and check the number of records

missing_seq = SeqIO.parse('output/sequences_missing_in_AmtDB_404.fasta', 'fasta')
missing_seq_metadata = pd.read_csv('output/metadata_for_sequences_missing_in_AmtDB_404.csv')

not_present_seq = SeqIO.parse('output/sequences_not_present_in_AmtDB_2996.fasta', 'fasta')
not_present_seq_metadata = pd.read_csv('output/metadata_for_sequences_not_present_in_AmtDB_2996.csv')

print(f"Number of missing sequences: {len(list(missing_seq))}")
print(f"Number of metadata records for missing sequences: {len(missing_seq_metadata)}")

print(f"Number of sequences not present in AmtDB: {len(list(not_present_seq))}")
print(f"Number of metadata records for sequences not present in AmtDB: {len(not_present_seq_metadata)}")

Number of missing sequences: 404
Number of metadata records for missing sequences: 431
Number of sequences not present in AmtDB: 3004
Number of metadata records for sequences not present in AmtDB: 2854


# EIGENSTRAT


software/EIG/bin/smartpca.perl -i data/mitogenomes_reich/v54.1.p1_HO_public/v54.1.p1_HO_public.geno -a data/mitogenomes_reich/v54.1.p1_HO_public/v54.1.p1_HO_public.snp -b data/mitogenomes_reich/v54.1.p1_HO_public/v54.1.p1_HO_public.ind -k 10 -o output/v54.1.p1_HO_public.pca -p output/v54.1.p1_HO_public.plot -e output/v54.1.p1_HO_public.eval -l output/v54.1.p1_HO_public.log